In [1]:
from pathlib import Path
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.pyplot as plt
import numpy as np
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %matplotlib inline
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
import pandas as pd 
import geopandas as gpd
import pygeos
import seaborn as sns
import xarray as xr
import rioxarray as rio
from matplotlib.colors import LinearSegmentedColormap
import datetime
from datetime import timedelta
from scipy.spatial.distance import cdist
from math import dist
from scipy.stats import ttest_ind
from geopy import distance
from scipy import stats
from matplotlib.gridspec import GridSpec
# from pylr2 import regress2
from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)
from matplotlib.ticker import MaxNLocator,FixedLocator
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde

# import shutil

/Users/weitao/anaconda3/lib/python3.10/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Match Pandora with concurrent GEMS

GEMS HCHO Version 2 data can be requested from the Korean Environmental Satellite Center (https://nesc.nier.go.kr/)

In [2]:
pandora4 = pd.read_pickle('Pandora_fuh5_MD2')
# pandora4 = pd.read_pickle('Pandora20240417min15')
pandora4

,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime
0,0.000040,36.7769,126.4938,1,2021010101,Seosan,2021-01-01 01:45:00
1,0.000047,37.4580,126.9510,1,2021010101,Seoul-SNU,2021-01-01 01:45:00
2,0.000039,36.7769,126.4938,2,2021010102,Seosan,2021-01-01 02:45:00
3,0.000080,37.4580,126.9510,2,2021010102,Seoul-SNU,2021-01-01 02:45:00
4,0.000035,36.7769,126.4938,3,2021010103,Seosan,2021-01-01 03:45:00
...,...,...,...,...,...,...,...
72460,0.000050,36.0513,140.1210,22,2023123122,Tsukuba-NIES,2023-12-31 22:45:00
72462,0.000039,36.0661,140.1244,23,2023123123,Tsukuba,2023-12-31 23:45:00
72463,0.000036,36.0513,140.1210,23,2023123123,Tsukuba-NIES,2023-12-31 23:45:00
72461,0.000030,35.6200,139.3834,23,2023123123,Tokyo-TMU,2023-12-31 23:45:00


In [3]:
target = list(pandora4['time'].unique())
print(len(target))
target[:5]

10360


[2021010101, 2021010102, 2021010103, 2021010104, 2021010200]

In [4]:
# define function 
def matchGEMS(tt):

    gemsVCDlis = []
    AMFCloudFractionlis = []
    SolarZenithAnglelis = []
    ViewingZenithAnglelis = []
    ulis = []
    vlis = []
    Wind_velocitylis = []
    vcd0Lis = []
    try:
        gems = pd.read_pickle('/media/dell/Elements1/downwind_code/data/L2_pickle_W_1000/'+str(tt)[8:10]+'45/'+str(tt)[0:8])
        temp = pandora4[pandora4['time']==tt].copy()

        for ind in temp.index:
        #     try:
            point = (temp.loc[ind,'Lat'], temp.loc[ind,'Lon'])
            buffer = 0.05
            gems1 = gems[(gems['Latitude']<point[0]+buffer) & (gems['Latitude']>point[0]-buffer) & (gems['Longitude']>point[1]-buffer) & (gems['Longitude']<point[1]+buffer)].copy()
            gems1 = gems1[gems1['AMFCloudFraction']<0.3]
        #     gems1 = gems1[gems1['SolarZenithAngle']<60]
            gems1 = gems1[gems1['FinalAlgorithmFlags']==0]
            gems1['VCD0'] = gems1['ColumnAmount'] - gems1['DSCD']/gems1['AirMassFactor']

            if (len(gems1['Latitude'])>0):

                weightLis = []
                for i in range(len(gems1)):
                    weight=1/distance.distance(point, (gems1.iloc[i]['Latitude'],gems1.iloc[i]['Longitude'])).km
                    weightLis.append(weight)
                summ = sum(weightLis)
                weight = [a/summ for a in weightLis]
                gems1['weight'] = weight
                gems1['weightedVCD']=gems1['weight']*gems1['ColumnAmount']/1e15
                gems1['weightedVCD0']=gems1['weight']*gems1['VCD0']/1e15
        #         gems['weightedError']=gems['weight']*gems['VCDError']/1e15
                aaa = gems1[['AMFCloudFraction','SolarZenithAngle','ViewingZenithAngle','u','v','Wind_velocity']].mean().to_list()

                gemsVCDlis.append(gems1['weightedVCD'].sum())
                AMFCloudFractionlis.append(aaa[0])
                SolarZenithAnglelis.append(aaa[1])
                ViewingZenithAnglelis.append(aaa[2])
                ulis.append(aaa[3])
                vlis.append(aaa[4])
                Wind_velocitylis.append(aaa[5])
                vcd0Lis.append(gems1['weightedVCD0'].sum())

            else:
                gemsVCDlis.append(-1000)
                AMFCloudFractionlis.append(-1000)
                SolarZenithAnglelis.append(-1000)
                ViewingZenithAnglelis.append(-1000)
                ulis.append(-1000)
                vlis.append(-1000)
                Wind_velocitylis.append(-1000)
                vcd0Lis.append(-1000)
                
        temp['GEMS'] = gemsVCDlis
        temp['AMFCloudFraction'] = AMFCloudFractionlis
        temp['SolarZenithAngle'] = SolarZenithAnglelis
        temp['ViewingZenithAngle'] = ViewingZenithAnglelis
        temp['u'] = ulis
        temp['v'] = vlis
        temp['Wind_velocity'] = Wind_velocitylis
        temp['vcd0'] = vcd0Lis
        temp = temp[temp['GEMS']!=-1000]
        print('tt')
        if(len(temp['GEMS'])>0):
            temp.to_pickle('./Hourly_MD/'+str(tt)[8:10]+str(tt)[0:8])
#         temp
    except:
        logm = open('./failed.txt','a')
        logm.write(str(tt)+" failed \n")
        logm.close()

In [ ]:
# Batch Processing
from joblib import Parallel, delayed
from tqdm import tqdm 
Parallel(n_jobs=12)(delayed(matchGEMS)(iday) for iday in tqdm(target));

  5%|█▌                                | 468/10360 [1:23:01<29:40:54, 10.80s/it]